# Little Lemon Sales Report
Welcome to the Lemon Sales report in Jupyter Notebook! The purpose of this project is to establish a database from a restaurant stored data and generate analysis from the database.

The restaurant name is Little Lemon, it is a family-owned Medierranian restaurant. They want to enable data-driven decisions to increase their revenue by having regular report of their business performance. They want the system to be easily maintainable, and easy to automate for further analysis.

The approach in this project is to develop a notebook that will setup a MySQL database, populate existing data, and generate some insights from it. They can run this notebook in their server and will having MySQL database automatically installed in their server.

This project has 5 parts:
**Step 1**: Setting up the database service
**Step 2**: Setting up the database schema
**Step 3**: Populate the existing data into the database
**Step 4**: Create Stored Procedures
**Step 5**: Analysis and report


## STEP 1: Setting up the Database Service
The first step is to setup the database, the database service used for the analysis is mysql.

In [1]:
# Install the mysql-connector-python package if not already installed
!pip install mysql-connector-python

  Using cached mysql_connector_python-9.4.0-py2.py3-none-any.whl.metadata (7.3 kB)
Using cached mysql_connector_python-9.4.0-py2.py3-none-any.whl (406 kB)


In [2]:
# Import the mysql.connector module
import mysql.connector as connector

# Connect to the MySQL database
try:
    connection = connector.connect(
        user='testuser',
        password='testpass',
    )
    if connection.is_connected():
        print("Successfully connected to the database")
except connector.Error as err:
    print(f"Error: {err}")

Successfully connected to the database


In [3]:
# Create a cursor object to interact with the database
try:
    cursor = connection.cursor()
    print("Cursor created successfully")
except connector.Error as err:
    print(f"Error: {err}")

Cursor created successfully


The database has successfully connected, as well as the cursor object is created. We can move forward to creating the database.

## STEP 2: Setting up the database
At this step we're going to setting up the little_lemon database as well as the schema of the database

In [4]:
# Create a new database named 'little_lemon'
try:
    cursor.execute("CREATE DATABASE IF NOT EXISTS little_lemon")
    print("Database 'little_lemon' created or already exists")
except connector.Error as err:
    print(f"Error: {err}")

Database 'little_lemon' created or already exists


In [5]:
# Set the database to use
try:
    cursor.execute("USE little_lemon")
    print("Using database 'little_lemon'")
except connector.Error as err:
    print(f"Error: {err}")

Using database 'little_lemon'


In [10]:
# Create MenuItems table
drop_menuitems_table_query = "DROP TABLE IF EXISTS MenuItems" # reset the table if it exists
create_menuitems_table_query = """
    CREATE TABLE IF NOT EXISTS MenuItems (
        ItemID INT AUTO_INCREMENT,
        Name VARCHAR(200),
        Type VARCHAR(100),
        Price INT,
        PRIMARY KEY (ItemID)
        );
    """
try:
    cursor.execute(drop_menuitems_table_query)
    cursor.execute(create_menuitems_table_query)
    print("Table 'MenuItems' created or already exists")
except connector.Error as err:
    print(f"Error: {err}")

Table 'MenuItems' created or already exists


In [13]:
# Create menu table
drop_menu_table_query = "DROP TABLE IF EXISTS Menus" # reset the table if it exists
create_menu_table_query = """
    CREATE TABLE IF NOT EXISTS Menus (
        MenuID INT,
        ItemID INT,
        Cuisine VARCHAR(100),
        PRIMARY KEY (MenuID, ItemID)
    );
    """
try:
    cursor.execute(drop_menu_table_query)
    cursor.execute(create_menu_table_query)
    print("Table 'Menu' created or already exists")
except connector.Error as err:
    print(f"Error: {err}")

Table 'Menu' created or already exists


In [14]:
# Create booking table
drop_booking_table_query = "DROP TABLE IF EXISTS Bookings" # reset the table if it exists
create_booking_table_query = """
    CREATE TABLE IF NOT EXISTS Bookings (
        BookingID INT AUTO_INCREMENT,
        TableNo INT,
        GuestFirstName VARCHAR(100) NOT NULL,
        GuestLastName VARCHAR(100) NOT NULL,
        BookingSlot TIME NOT NULL,
        EmployeeID INT,
        PRIMARY KEY (BookingID)
    );
    """
try:
    cursor.execute(drop_booking_table_query)
    cursor.execute(create_booking_table_query)
    print("Table 'Bookings' created or already exists")
except connector.Error as err:
    print(f"Error: {err}")

Table 'Bookings' created or already exists


In [16]:
# Create orders table
drop_orders_table_query = "DROP TABLE IF EXISTS Orders" # reset the table if it exists
create_orders_table_query = """
    CREATE TABLE IF NOT EXISTS Orders (
        OrderID INT,
        TableNo INT,
        MenuID INT,
        BookingID INT,
        BillAmount INT,
        Quantity INT,
        PRIMARY KEY (OrderID, TableNo)
    );
    """
try:
    cursor.execute(drop_orders_table_query)
    cursor.execute(create_orders_table_query)
    print("Table 'Orders' created or already exists")
except connector.Error as err:
    print(f"Error: {err}")

Table 'Orders' created or already exists


In [17]:
# Create employees table
drop_employees_table_query = "DROP TABLE IF EXISTS Employees" # reset the table if it exists
create_employees_table_query = """
    CREATE TABLE IF NOT EXISTS Employees (
        EmployeeID INT AUTO_INCREMENT PRIMARY KEY,
        Name VARCHAR(255),
        Role VARCHAR(100),
        Address VARCHAR(255),
        Contact_Number INT,
        Email VARCHAR(255),
        Annual_Salary VARCHAR(100)
        );
    """
try:
    cursor.execute(drop_employees_table_query)
    cursor.execute(create_employees_table_query)
    print("Table 'Employees' created or already exists")
except connector.Error as err:
    print(f"Error: {err}")

Table 'Employees' created or already exists


## STEP 3: Populate the data into the database
At this step we're going to populate the available data from the restaurant into the database

In [31]:
#*******************************************************#
# Insert query to populate "MenuItems" table:
#*******************************************************#
insert_menuitems="""
INSERT INTO MenuItems (ItemID, Name, Type, Price)
VALUES
(1, 'Olives','Starters',5),
(2, 'Flatbread','Starters', 5),
(3, 'Minestrone', 'Starters', 8),
(4, 'Tomato bread','Starters', 8),
(5, 'Falafel', 'Starters', 7),
(6, 'Hummus', 'Starters', 5),
(7, 'Greek salad', 'Main Courses', 15),
(8, 'Bean soup', 'Main Courses', 12),
(9, 'Pizza', 'Main Courses', 15),
(10, 'Greek yoghurt','Desserts', 7),
(11, 'Ice cream', 'Desserts', 6),
(12, 'Cheesecake', 'Desserts', 4),
(13, 'Athens White wine', 'Drinks', 25),
(14, 'Corfu Red Wine', 'Drinks', 30),
(15, 'Turkish Coffee', 'Drinks', 10),
(16, 'Turkish Coffee', 'Drinks', 10),
(17, 'Kabasa', 'Main Courses', 17);"""

#*******************************************************#
# Insert query to populate "Menu" table:
#*******************************************************#
insert_menu="""
INSERT INTO Menus (MenuID,ItemID,Cuisine)
VALUES
(1, 1, 'Greek'),
(1, 7, 'Greek'),
(1, 10, 'Greek'),
(1, 13, 'Greek'),
(2, 3, 'Italian'),
(2, 9, 'Italian'),
(2, 12, 'Italian'),
(2, 15, 'Italian'),
(3, 5, 'Turkish'),
(3, 17, 'Turkish'),
(3, 11, 'Turkish'),
(3, 16, 'Turkish');"""

#*******************************************************#
# Insert query to populate "Bookings" table:
#*******************************************************#
insert_bookings="""
INSERT INTO Bookings (BookingID, TableNo, GuestFirstName, 
GuestLastName, BookingSlot, EmployeeID)
VALUES
(1, 12, 'Anna','Iversen','19:00:00',1),
(2, 12, 'Joakim', 'Iversen', '19:00:00', 1),
(3, 19, 'Vanessa', 'McCarthy', '15:00:00', 3),
(4, 15, 'Marcos', 'Romero', '17:30:00', 4),
(5, 5, 'Hiroki', 'Yamane', '18:30:00', 2),
(6, 8, 'Diana', 'Pinto', '20:00:00', 5);"""

#*******************************************************#
# Insert query to populate "Orders" table:
#*******************************************************#
insert_orders="""
INSERT INTO Orders (OrderID, TableNo, MenuID, BookingID, Quantity, BillAmount)
VALUES
(1, 12, 1, 1, 2, 86),
(2, 19, 2, 2, 1, 37),
(3, 15, 2, 3, 1, 37),
(4, 5, 3, 4, 1, 40),
(5, 8, 1, 5, 1, 43);"""

#*******************************************************#
# Insert query to populate "Employees" table:
#*******************************************************#
insert_employees = """
INSERT INTO Employees (EmployeeID, Name, Role, Address, Contact_Number, Email, Annual_Salary)
VALUES
(01,'Mario Gollini','Manager','724, Parsley Lane, Old Town, Chicago, IL',351258074,'Mario.g@littlelemon.com','$70,000'),
(02,'Adrian Gollini','Assistant Manager','334, Dill Square, Lincoln Park, Chicago, IL',351474048,'Adrian.g@littlelemon.com','$65,000'),
(03,'Giorgos Dioudis','Head Chef','879 Sage Street, West Loop, Chicago, IL',351970582,'Giorgos.d@littlelemon.com','$50,000'),
(04,'Fatma Kaya','Assistant Chef','132  Bay Lane, Chicago, IL',351963569,'Fatma.k@littlelemon.com','$45,000'),
(05,'Elena Salvai','Head Waiter','989 Thyme Square, EdgeWater, Chicago, IL',351074198,'Elena.s@littlelemon.com','$40,000'),
(06,'John Millar','Receptionist','245 Dill Square, Lincoln Park, Chicago, IL',351584508,'John.m@littlelemon.com','$35,000');"""


In [ ]:
# Populate MenuItems table
cursor.execute("DELETE FROM MenuItems") # reset the table if it exists
cursor.execute(insert_menuitems)
connection.commit()

# Populate MenuItems table
cursor.execute("DELETE FROM Menus") # reset the table if it exists
cursor.execute(insert_menu)
connection.commit()

# Populate Bookings table
cursor.execute("DELETE FROM Bookings") # reset the table if it exists
cursor.execute(insert_bookings)
connection.commit()

# Populate Orders table
cursor.execute("DELETE FROM Orders") # reset the table if it exists
cursor.execute(insert_orders)
connection.commit()

# Populate Employees table
cursor.execute("DELETE FROM Employees") # reset the table if it exists
cursor.execute(insert_employees)
connection.commit()

# Close the cursor and connection
cursor.close()
connection.close()

## STEP 4: Create Stored Procedures

In [173]:
# Create mysql connection pool
from mysql.connector.pooling import MySQLConnectionPool
from mysql.connector import Error

dbconfig = {
    "user": "testuser",
    "password": "testpass",
    "database": "little_lemon"
}

try:
    pool = MySQLConnectionPool(
        pool_name="mypool",
        pool_size=2,
        **dbconfig
    )
    print("Connection pool created successfully")
except Error as err:
    print(f"Error code: {err.errno}")
    print(f"SQLSTATE: {err.sqlstate}")
    print(f"Error Message: {err.msg}")

Connection pool created successfully


In [174]:
# Get connection from the pool
try:
    connection1 = pool.get_connection()
    if connection1.is_connected():
        print("'connection1' object is created successfully from the pool")
except Error as err:
    print(f"Error: {err}")

# Create a cursor object using the connection from the pool
try:
    cursor1 = connection1.cursor()
    print("Cursor created successfully from the connection pool")
except Error as err:
    print(f"Error: {err}")

'connection1' object is created successfully from the pool
Cursor created successfully from the connection pool


In [175]:
# Inspect the connection pool properties
print(f"Connection Pool Name: {pool.pool_name}")
print(f"Connection Pool Size: {pool.pool_size}")
print(f"Active Connections: {pool._cnx_queue.qsize()}")

# Execute a query using the connection from the pool for testing
try:
    cursor1.execute("SELECT * FROM Bookings ORDER BY BookingSlot DESC")
    bookings = cursor1.fetchall()
    print(bookings)
except Error as err:
    print(f"Error: {err}")


Connection Pool Name: mypool
Connection Pool Size: 2
Active Connections: 1
[(6, 8, 'Diana', 'Pinto', datetime.timedelta(seconds=72000), 5), (1, 12, 'Anna', 'Iversen', datetime.timedelta(seconds=68400), 1), (2, 12, 'Joakim', 'Iversen', datetime.timedelta(seconds=68400), 1), (5, 5, 'Hiroki', 'Yamane', datetime.timedelta(seconds=66600), 2), (4, 15, 'Marcos', 'Romero', datetime.timedelta(seconds=63000), 4), (3, 19, 'Vanessa', 'McCarthy', datetime.timedelta(seconds=54000), 3)]


In [ ]:
# Implement a stored procedure called PeakHours
create_procedure_peak_hours = """
    DROP PROCEDURE IF EXISTS PeakHours;
    CREATE PROCEDURE PeakHours()
    BEGIN
        SELECT HOUR(BookingSlot) AS Booking_Hour, COUNT(*) AS NumberOfBookings
        FROM Bookings
        GROUP BY Booking_Hour
        ORDER BY NumberOfBookings DESC;
    END;
    """

# Execute the stored procedure creation query, and fetch the results
cursor1.execute(create_procedure_peak_hours)
cursor1.callproc('PeakHours')

results = cursor1.stored_results()
data = results.fetchall()

# Retrieve column names using list comprehension in a for loop 
for column in cursor.stored_results():
    columns = [column[0] for column in column_id.description]

print(columns)

# Print data 
for row in data:
    print(row)

connection1.commit()

Booking Hour    Number of Bookings  
19              2                   
15              1                   
17              1                   
18              1                   
20              1                   


In [ ]:
guest_status_stored_procedure = """
    CREATE PROCEDURE GuestStatus()

    BEGIN

    SELECT 

    Bookings.BookingID AS OrderNumber,  
    CONCAT(GuestFirstName,' ',GuestLastName) AS GuestName, 

    Role AS Employee, 

    CASE 
    WHEN Role IN ('Manager','Assistant Manager') THEN "Ready to Pay"
    WHEN Role = 'Head Chef' THEN "Ready to serve"
    WHEN Role = 'Assistant Chef' THEN "Preparing order"
    WHEN Role = 'Head Waiter' THEN "Order served"

    ELSE "Pending"
    END AS Status

    FROM Bookings 
    LEFT JOIN 
    Employees 
    ON Employees.EmployeeID=Bookings.EmployeeID;

    END
"""



# Execute the query and fetch the results
cursor.execute(stored_procedure_query)
cursor.callproc("GuestStatus")

results = next(cursor.stored_results())
guests = results.fetchall()

# Retrieve column names using list comprehension in a for loop 
for column_id in cursor.stored_results():
    columns = [column[0] for column in column_id.description]

# Print column names
print(columns)

# Print data 
for guest in guests:
    print(guest)

connection1.commit()
# close th cursor and connections
cursor1.close()
connection1.close()

Order Number    Guest Name                Employee             Status              


## STEP 5: Little Lemon Analysis and Sales Report

In [179]:
# Import MySQLConnectionPool class
from mysql.connector.pooling import MySQLConnectionPool

# Import Error class
from mysql.connector import Error

# Define database configurations
dbconfig = {
    "database":"little_lemon",
    "user" : "root",
    "password" : "rootpass"
}

# Create a pool named "pool_b" with two connections.
# try-except block
try:
    pool = MySQLConnectionPool(pool_name = "mypool_a",pool_size = 2,**dbconfig)
    print("The connection pool is created with the name: ",pool.pool_name)
    print("The pool size is:",pool.pool_size)

except Error as er:
    print("Error code:", er.errno)
    print("Error message:", er.msg)


The connection pool is created with the name:  mypool_a
The pool size is: 2


In [180]:
# Connect the first guest.
connection1 = pool.get_connection()
cursor1=connection1.cursor()

booking1_query="""
    INSERT INTO Bookings 
    (TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID)
    VALUES
    (8,'Anees','Java','18:00:00',6);
"""
# Execute the insert query
cursor1.execute(booking1_query)
connection1.commit()
print("""A new booking is added in the "Bookings" table.""")



A new booking is added in the "Bookings" table.


In [181]:
# Connect the second guest .
connection2 = pool.get_connection()
cursor2=connection2.cursor()

booking2_query="""
    INSERT INTO Bookings 
    (TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID)
    VALUES
    (5, 'Bald','Vin','19:00:00',6);
    """

cursor2.execute(booking2_query)
connection2.commit()

print("""A new booking is added in the "Bookings" table.""")


A new booking is added in the "Bookings" table.


In [183]:
# Adding a new connection to connect the third user.
try:
    connection3 = pool.get_connection()
    print("The guest is connected")
except:
    print("Adding new connection in the pool.")
        
    # Create a connection
    connection=connector.connect(user="testuser",password="testpass")
    # Add the connection into the pool
    pool.add_connection(cnx=connection)
    print("A new connection is added in the pool.\n")
        
    connection3 = pool.get_connection()
    print("'connection3' is added in the pool.")
    
# Connect the third guest 
cursor3 = connection3.cursor()

booking3_query = """
    INSERT INTO Bookings 
    (TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID)
    VALUES
    (12, 'Jay','Kon','19:30:00',6);
    """

cursor3.execute(booking3_query)
connection3.commit()

print("""A new booking is added in the "Bookings" table.""")

#  Close all  connections.

for connection in [connection1,connection2, connection3]:
    try:
        connection.close()
        print("Connection is returned to the pool")
    except Error as er:
        print("\nConnection can't be returned to the pool")
        print("Error message:", er.msg)


Adding new connection in the pool.
A new connection is added in the pool.

'connection3' is added in the pool.
A new booking is added in the "Bookings" table.
Connection is returned to the pool
Connection is returned to the pool

Connection can't be returned to the pool
Error message: Failed adding connection; queue is full


In [184]:
# Get a connection from mypool and create a cursor object 
connection = pool.get_connection()
print("""The object "connection" is created with a connection link from the mypool""")

cursor = connection.cursor()
print("""The cursor object "cursor" is created.""")



The object "connection" is created with a connection link from the mypool
The cursor object "cursor" is created.


In [185]:
# Query the Little Lemon manager.
cursor.execute("""
    SELECT 
    Name, EmployeeID 
    FROM Employees 
    WHERE Role = 'Manager';
    """)

results=cursor.fetchall()
columns=cursor.column_names

print(columns)
for result in results:
    print(result)



('Name', 'EmployeeID')
('Mario Gollini', 1)


In [186]:
# Query the name and role of the employee who receives the highest salary
cursor.execute("""
    SELECT Name, EmployeeID 
    FROM Employees ORDER BY 
    Annual_Salary DESC LIMIT 1
    """)

results=cursor.fetchall()
columns=cursor.column_names

print(columns)

for result in results:
    print(result)



('Name', 'EmployeeID')
('Mario Gollini', 1)


In [187]:
# Query the number of guests booked between 6 - 8 PM.
cursor.execute("""SELECT 
COUNT(BookingID) n_booking_between_18_20_hrs
FROM Bookings 
WHERE BookingSlot BETWEEN '18:00:00' AND '20:00:00';""")
results=cursor.fetchall()
columns=cursor.column_names
print(columns)
for result in results:
    print(result)

('n_booking_between_18_20_hrs',)
(7,)


In [188]:
# Query the full name and the BookingId of each guest waiting to be seated  
# with the receptionist in sorted order with respect to their BookingSlot.

cursor.execute("""
    SELECT 

    Bookings.BookingID AS ID,  
    CONCAT(GuestFirstName,' ',GuestLastName) AS GuestName, 

    Role AS Employee

    FROM Bookings 
    LEFT JOIN 
    Employees 
    ON Employees.EmployeeID=Bookings.EmployeeID
    WHERE Employees.Role = "Receptionist"
    ORDER BY BookingSlot DESC;
""")

print("The following guests are waiting to be seated:")

results=cursor.fetchall()
columns=cursor.column_names

print(columns)

for result in results:
    print(result)


The following guests are waiting to be seated:
('ID', 'GuestName', 'Employee')
(9, 'Jay Kon', 'Receptionist')
(8, 'Bald Vin', 'Receptionist')
(7, 'Anees Java', 'Receptionist')


In [ ]:
# Create and execute a stored procedure named BasicSalesReport. 
cursor.execute("DROP PROCEDURE IF EXISTS BasicSalesReport;")
stored_procedure_query="""
CREATE PROCEDURE BasicSalesReport()

BEGIN
SELECT 
SUM(BillAmount) AS Total_Sale,
AVG(BillAmount) AS Average_Sale,
MIN(BillAmount) AS Min_Bill_Paid,
MAX(BillAmount) AS Max_Bill_Paid
FROM Orders;
END
"""


# Execute the query and call the stored procedure with its name
cursor.execute(stored_procedure_query)
cursor.callproc("BasicSalesReport")

# Retrieve records in "report"
results = next(cursor.stored_results())
report = results.fetchall()

# Retrieve column names using list comprehension in a for loop 
for column_id in cursor.stored_results():
    cols = [column[0] for column in column_id.description]

    
print("Today's sales report:")
for row in report:
    print("\t",cols[0],"":"",row[0])
    print("\t",cols[1],"":"",row[1])
    print("\t",cols[2],"":"",row[2])
    print("\t",cols[3],"":"",row[3])

connection.commit()

Today's sales report:
	 Total_Sale : 243
	 Average_Sale : 48.6000
	 Min_Bill_Paid : 37
	 Max_Bill_Paid : 86


In [193]:
# Query the booking slot, guest name, assigned employee name and role 
# for the first three bookings in ascending order of their booking slot.
booking_slot_query="""
    SELECT 
    Bookings.BookingSlot,
    CONCAT(Bookings.GuestFirstName," ",Bookings.GuestLastName) AS Guest_Name,
    Employees.Name AS Emp_Name,
    Employees.Role AS Emp_Role
    FROM Bookings 
    INNER JOIN 
    Employees ON Bookings.EmployeeID=Employees.EmployeeID
    ORDER BY Bookings.BookingSlot ASC;
    """

# Execute the query and fetch the results
cursor.execute(booking_slot_query)
results=cursor.fetchmany(size=3)

for result in results:
    print("\nBookingSlot",result[0])
    print("\tGuest_name:",result[1])
    print("\tAssigned to:",result[2],"[{}]".format(result[3]))
    
connection.close()


BookingSlot 15:00:00
	Guest_name: Vanessa McCarthy
	Assigned to: Giorgos Dioudis [Head Chef]

BookingSlot 17:30:00
	Guest_name: Marcos Romero
	Assigned to: Fatma Kaya [Assistant Chef]

BookingSlot 18:00:00
	Guest_name: Anees Java
	Assigned to: John Millar [Receptionist]
